In [3]:
"""
功能说明：
1. 利用QRCode区域，做kmeans 2分类，找到最有可能是蓝色的中心值
2. 计算每张图片的hsv和qrcode的中心值，送入gradientboost，做一个G通道和R通道的预测
3. 根据预测结果，做bitmask，只保留敏感区域，其他部分涂黑
4. 计算x方向梯度，做二值化，去噪点，放大白色区域
5. 利用这个结果做边缘检测
6. 通过merge_boxes，选择在水平或者垂直方向做Box的合并
7. 按照合并后的box输出原始图片
8. 去除掉原始图片中非常明显的红色区域
"""
# -*- coding: utf-8 -*-
import cv2,math,glob
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
%matplotlib inline
import copy
from scipy.spatial import distance
import datetime
from darwinutils.mapreduce import parallel_starmap_p

source_path = 'image/origin'
qrcode_path = 'image/qrcode'
target_path = 'image/output'
tmp_path = 'image/tmp'


def get_blue_color_center_by_kmeans(img, cluster_num):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv2.KMEANS_RANDOM_CENTERS
    K= cluster_num
    Z = img.reshape((-1,np.shape(img)[-1]))
    # convert to np.float32
    Z = np.float32(Z)
    ret,label,center = cv2.kmeans(Z,K,None,criteria,5,cv2.KMEANS_RANDOM_CENTERS) #KMEANS_PP_CENTERS
    #print(center)
    max_ratio = 0
    max_ratio_color_idx = 0
    b_channel = 0
    g_channel = 1
    r_channel = 2
    for idx, color in enumerate(center):
        color_ratio = sum([float(color[b_channel])/float(color[g_channel])+0.00001, 
                           float(color[b_channel])/float(color[r_channel])+0.00001])
        if color_ratio > max_ratio:
            max_ratio = color_ratio
            max_ratio_color_idx = idx
    rst = np.array(center[max_ratio_color_idx], dtype=int)    
    return rst

def get_hsv_stat(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mean = np.mean(hsv)
    mean_v = np.mean(hsv[:,:,2])
    return mean, mean_v

def _parallel_update_img(img_row, row_width, threshold_g, threshold_r, value, origin_img_flag):
    if isinstance(value, (list, np.ndarray)):
        if origin_img_flag:
            processed = img_row
        else:
            processed = np.full((row_width, len(value)), np.full((len(value)), 255, dtype=np.uint8), dtype=np.uint8)
    else:
        if origin_img_flag:
            processed = img_row
        else:
            processed = np.full((row_width), 255, dtype=np.uint8)
    for idx in range(row_width):
        if img_row[idx][0] <= img_row[idx][1]+threshold_g or img_row[idx][0] <= img_row[idx][2]+threshold_r:
            processed[idx] = value
    return processed
    
def parallel_update_img(img, threshold_g, threshold_r, value, origin_img_flag=False):
    img_h,img_w,img_c = img.shape
    param_lst = []
    for idx in range(img_h):
        param_lst.append([img[idx], img_w, threshold_g, threshold_r, value, origin_img_flag])
    rst = parallel_starmap_p(_parallel_update_img, param_lst)
    rst = np.array(list(rst), dtype=np.uint8)
    return rst
    
def get_img_by_specified_blue_color(img, feature):    
    img_h,img_w,img_c = img.shape
    print(feature)
    threshold_g = round(model_g.predict([feature])[0])
    threshold_r = round(model_r.predict([feature])[0])
    print("{} - {}".format(threshold_g, threshold_r))
    processed = parallel_update_img(img, threshold_g, threshold_r, 0)

    output = cv2.bitwise_and(img, img, mask=processed)
    #cv2.imwrite("t2.jpg",output)  

    return output, threshold_g, threshold_r

def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = (xB - xA + 1) * (yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

def merge_two_boxes(boxA, boxB):
    xmin = min(boxA[0], boxB[0])
    ymin = min(boxA[1], boxB[1])
    xmax = max(boxA[2], boxB[2])
    ymax = max(boxA[3], boxB[3])
    return [xmin, ymin, xmax, ymax]
    
def merge_boxes(boxes, direction="horizon"):
    # sort by direction
    work_boxes = copy.copy(boxes)
    work_boxes = np.array(work_boxes)
    box_margin_threshold = 10
    box_extern_margin = 30
    if direction == "horizon":
        #work_boxes = sorted(work_boxes, key=lambda _: (_[0], _[1], _[2]))
        direction_idx_lst = [1, 3]
    else:
        #work_boxes = sorted(work_boxes, key=lambda _: (_[1], _[0], _[3]))
        direction_idx_lst = [0, 2]
    
    new_boxes_lst = []
    #print(len(work_boxes))
    while len(work_boxes) > 1:          
        current_box = work_boxes[0]
        tt = copy.copy(current_box)
        merged_box_idx = []
        #if abs(tt[1] - 505) <=5 and abs(tt[0] - 1530) <= 85:
        #print("PPP1: {} - {} - {}".format(len(work_boxes), current_box, tt))
        for next_box_idx, next_box in enumerate(work_boxes[1:]):
            #if abs(tt[1] - 274) <=50 and abs(tt[0] - 1246) <= 25:
            #print("AA1: {} - {} - {}".format(len(work_boxes), current_box, tt))   
            if abs(current_box[direction_idx_lst[0]] - next_box[direction_idx_lst[0]]) <= box_margin_threshold or abs(
                current_box[direction_idx_lst[1]] - next_box[direction_idx_lst[1]]) <= box_margin_threshold or (
                    current_box[direction_idx_lst[0]] > next_box[direction_idx_lst[0]] and current_box[direction_idx_lst[1]] < next_box[
                direction_idx_lst[1]]) or (
                    current_box[direction_idx_lst[0]] < next_box[direction_idx_lst[0]] and current_box[direction_idx_lst[1]] > next_box[
                direction_idx_lst[1]]):                  
                boxA = copy.copy(current_box)
                boxB = copy.copy(next_box)
                
                #if abs(tt[1] - 274) <=50 and abs(tt[0] - 1246) <= 25:
                #    print("BB1: {} - {} - {}".format(len(work_boxes), current_box, next_box))
                
                if boxA[1 - direction_idx_lst[0]] < boxB[1 - direction_idx_lst[0]]:
                    boxA[5 - direction_idx_lst[1]] += box_extern_margin
                    boxB[1 - direction_idx_lst[0]] -= box_extern_margin
                else:
                    boxB[5 - direction_idx_lst[1]] += box_extern_margin
                    boxA[1 - direction_idx_lst[0]] -= box_extern_margin
                
                if bb_intersection_over_union(boxA, boxB) > 0:
                    #merge
                    current_box = merge_two_boxes(current_box, next_box)
                    merged_box_idx.append(1+next_box_idx)
        #if abs(tt[1] - 505) <=5 and abs(tt[0] - 1530) <= 85:
        #print("CC: {} - {}".format(len(work_boxes), current_box))  
        if len(merged_box_idx) == 0:
            new_boxes_lst.append(current_box)
            work_boxes = np.delete(work_boxes, 0, axis=0)  
        else:
            work_boxes[0] = current_box
            work_boxes = np.delete(work_boxes, merged_box_idx, axis=0)                
            
    new_boxes_lst.append(work_boxes[0])
    return new_boxes_lst



def init_predict_mode():
    from sklearn.multioutput import MultiOutputRegressor
    from sklearn.ensemble import GradientBoostingRegressor
    X_train = np.array([
        [109.52697905020497, 251.44698730234788, 137.65395052379148, 240.71809920484665, 234, 180, 135],
        [116.17879851461429, 249.1167776712985, 131.20780380053256, 242.66412490922295, 237, 187, 183],
        [109.3761421231965, 250.44839522440506, 132.54682753383042, 239.5353148301574, 232, 195, 154],
        [99.58768766970132, 250.96150175690784, 141.83947074060993, 239.38029848110207, 231, 147, 100],
        [106.27835343129425, 250.36086927008466, 129.8668252080856, 242.76751486325801, 237, 189, 186],
        [70.38107038279296, 126.72922696054944, 84.69871794871794, 99.36625874125875, 87, 77, 60],
        [66.56835090241175, 122.32527112282384, 106.48280633574751, 96.30255255255256, 73, 44, 26],
        [71.10329566629399, 151.68015093435554, 75.30146620958267, 95.22350441794215, 81, 72, 62],
        [66.48773172549646, 133.13002954799552, 92.11390345765346, 110.23419289044288, 94, 75, 61],
        [68.68050763988713, 129.58216059734866, 101.29746233391636, 105.24233637116818, 92, 54, 35],
        [100.85336194963531, 243.5352870947133, 147.9085989767808, 184.23696379378197, 155,  40,  31],
        [111.70328807964648, 239.25448690305063, 142.53413333333333, 179.79308571428572, 142,  36,  29],
        [101.61245794069104, 242.9208437150615, 144.73899051490514, 188.7819660014782, 155,  42,  36],
        [103.18516397806528, 241.15553905126976, 135.5484945601754, 180.4998734924517, 139,  48,  40],
        [94.7470269658734, 207.40600143747005, 115.81971193415637, 181.1569135802469, 154, 120,  98],        
        #############################################################################################
        [91.26939000692116, 184.13203162434115, 126.30109212802768, 160.91657655709344, 135, 93, 53],
        [102.49516198157909, 242.30254032902093, 142.82239842454393, 188.9104995854063, 152, 46, 37],
        #############################################################################################
        [97.15168583293404, 247.02621306500558, 117.37486260454003, 229.5037634408602, 201, 149, 151],
        [99.04099158813821, 245.35120188468295, 128.12003856660775, 216.23214955273448, 174, 109, 89],
        #############################################################################################
        [98.64754711707396, 238.98480594154287, 126.13953940606162, 201.75068755729643, 149, 91, 71],
        [98.00318665814832, 248.273548554544, 124.31400279448393, 229.7188627665391, 209, 156, 156],
        [97.71518620561146, 207.9527212106692, 122.29488397877564, 176.84954858636254, 154, 126, 88],
        [103.61169075759996, 243.12885202044401, 143.3168342761366, 209.71087366436203, 180, 108, 55],
        [102.19240882713092, 243.32113719853058, 134.6360082304527, 227.3279541446208, 211, 112, 112], 
        [75.08717404035563, 156.3277631368791, 98.67653457247776, 150.38322978123531, 131, 114, 95],
        [101.30381901187243, 247.3142808656764, 134.3789697424356, 218.57670667666918, 204, 155, 130],
        [97.41451232497471, 248.79167105893626, 126.09593168477582, 231.8404833270888, 216, 171, 162],
        [96.39889354735665, 190.40718455518288, 121.71423841059602, 175.6060706401766, 156, 122, 89],
        [102.22164071234627, 242.45107850183678, 134.36488543502645, 187.31841852267266, 143, 56, 45],
        [102.9854357663845, 242.01362721609968, 138.00511256354395, 185.50296296296295, 142, 55, 42], 
        [106.2656987701645, 239.97597348666346, 145.42920692693212, 190.84912691191127, 151, 48, 36],
        [102.6273241761167, 243.57372783900337, 123.38869720784615, 219.8508721487445, 182, 138, 114],
        [113.62966259383485, 242.6542253633605, 141.05553613053613, 209.64283216783215, 189, 123, 99],
        [119.63441995421391, 243.20553026673056, 136.6059266598077, 223.11618336318256, 204, 135, 113],
        [97.77101341638716, 248.545106213065, 120.49058142150247, 230.53879892037787, 208, 175, 173],
        [106.620433237502, 242.12590760261938, 132.04369918699186, 197.74369071815718, 150, 74, 57],
        [112.37496659213119, 224.63844393866793, 131.9551706718626, 224.93024464169733, 208, 154, 148],
        [101.96412553905127, 245.14165748283023, 144.21596505040213, 202.7112490158848, 161, 50, 35],
        [98.95229595911196, 246.52794721290527, 135.53701013898382, 219.4862867395762, 196, 133, 99],
        [71.19230101687697, 128.26064007347068, 95.67592391304348, 115.41603260869566, 95, 81, 58],
        [85.2706077303945, 175.79466938188787, 93.25238095238095, 151.5040293040293, 142, 136, 117],
        [76.03268607251238, 155.07993531384764, 106.0945895121427, 167.19233558994196, 156, 128, 117],
        [99.02903449928127, 247.06253074588724, 138.41820728291316, 224.43051470588236, 212, 135, 132],
        [101.30381901187243, 247.3142808656764, 134.3789697424356, 218.57670667666918, 204, 155, 130],
        [72.58678259596444, 150.87245288292604, 101.92441304941305, 143.32995495495496, 130, 99, 80],
        [102.9854357663845, 242.01362721609968, 138.00511256354395, 185.50296296296295, 142, 55, 42],
        [102.6273241761167, 243.57372783900337, 123.38869720784615, 219.8508721487445, 182, 138, 113],
        [85.2706077303945, 175.79466938188787, 93.25238095238095, 151.5040293040293, 142, 135, 116],
        [76.03268607251238, 155.07993531384764, 106.0945895121427, 167.19233558994196, 156, 128, 117],
        [99.94074681893201, 242.67670579779588, 134.1112386621315, 204.58061224489796, 182, 119, 101],
        [81.55514108502369, 150.73797117073948, 91.2229283990346, 142.42172164119066, 124, 131, 102],
        [97.63948450726721, 249.50728318160037, 141.9881149000939, 224.87496356747303, 207, 121, 78],
        [84.89661036575627, 193.71843994569556, 98.60596914822267, 157.06572769953053, 128, 114, 100],
        [78.04891644039823, 146.25934714901774, 76.70062066447609, 125.2842278203724, 129, 134, 119],
        [89.39296158760581, 158.1669853058617, 114.52662315355548, 140.03538302988665, 116, 92, 55],
        [76.03268607251238, 155.07993531384764, 106.0945895121427, 167.19233558994196, 156, 128, 118],
        [84.67053106532502, 175.1478977000479, 131.3150319260256, 151.78409891729132, 127, 66, 34],
        [71.07405419794495, 146.84841998083374, 89.46794261943403, 137.7153729923331, 123, 116, 97],
        [69.6899108236171, 147.26727799073632, 106.04862493548625, 121.31707586817076, 111, 76, 59],
        [81.76003567055317, 134.8985689187031, 121.53121621621622, 116.14360360360361, 100, 47, 18],
        [81.3631817334824, 159.55710948730234, 98.14963784866697, 139.05409153952843, 124, 112, 89],
        [90.2021869509663, 183.09435074269285, 94.97237076648841, 192.11390374331552, 191, 194, 183],
        [104.15263682585316, 244.82142828621625, 139.95946078431373, 203.31357843137255, 168, 84, 57],
    ])

    y_train = np.array([
        [14, 47],
        [20, 22],
        [6, 49], 
        [13, 41],
        [11, 11],
        [4, 5],
        [0, 10],
        [0, -2],
        [-1, 5],
        [3, 11],
        [55, 56],
        [19, 37],
        [108, 106],
        [13, 19],
        [-4, 11],
        #####
        [-6, 7],
        [15, 31],
        #####
        [2, 11],
        [8, 11],
        #####
        [-1, 8],        
        [11, 11],
        [-8, 11],
        [10, 37],
        [13, 49],
        [-2.0, 11.0],
        [2, 17],
        [8, 11],
        [-3, 19],
        [13, 27],
        [13, 48],
        [27, 57],
        [0, 7],
        [19, 36],
        [15, 32],
        [3, 4],
        [9, 26],
        [16, 6],
        [24, 61],
        [6, 21],
        ########
        [-10, -7],
        [-7, -2],
        [-4, 5],
        [6, 16],
        [3, 4],
        [-2, 15],
        [6, 25],
        [0, 12],
        [-4, 2],
        [-4, 5],
        #######
        [7, 20],
        [-9, 12],
        [8, 24],
        [-11, 0],
        [-4, 6],
        [-5, 20],
        [1, 2],
        [2, 16],
        [-6, -5],
        [12, 10],
        [15, 28],
        [-10, 7],
        [-1, 3],
        [9, 12],
    ])
    max_depth = 20
    model_g = GradientBoostingRegressor(n_estimators=50, loss='huber', max_depth=max_depth)
    model_g.fit(X_train, y_train[:,0])
    model_r = GradientBoostingRegressor(n_estimators=50, loss='huber', max_depth=max_depth)
    model_r.fit(X_train, y_train[:,1])
    return model_g, model_r

In [6]:
source_file_lst = glob.glob(os.path.join(source_path, '*.jpg'))
model_g, model_r = init_predict_mode()
for source_file in source_file_lst:
    #if source_file.find("ZP0672") < 0:
    #    continue
    qrcode_file = "{}/{}_qrcode.jpg".format(qrcode_path, os.path.splitext(os.path.basename(source_file))[0])
    if not os.path.exists(qrcode_file):
        print("Missing qrcode jpg for image {}".format(source_file))
        continue
    print("Processing {}".format(os.path.basename(source_file)))
    full_img = cv2.imread(source_file,cv2.IMREAD_COLOR)
    qr_img = cv2.imread(qrcode_file,cv2.IMREAD_COLOR)
    before_time = datetime.datetime.now()
    blue_color = get_blue_color_center_by_kmeans(qr_img, 2)
    qr_hsv_mean, qr_hsv_v_mean = get_hsv_stat(qr_img)
    full_hsv_mean, full_hsv_v_mean = get_hsv_stat(full_img)
    after_time = datetime.datetime.now()
    #print("Kmeans Spend {}\n{}".format((after_time-before_time).seconds, blue_color))
    input_feature = [full_hsv_mean, full_hsv_v_mean, qr_hsv_mean, qr_hsv_v_mean, blue_color[0], blue_color[1], blue_color[2]]
    before_time = datetime.datetime.now()
    output, threashold_g, threashold_r = get_img_by_specified_blue_color(full_img, input_feature)
    after_time = datetime.datetime.now()
    print("Transfer Image Spend {}".format((after_time-before_time).seconds))
    
    tmp_file = "{}/{}_transfer.jpg".format(tmp_path, os.path.splitext(os.path.basename(source_file))[0])
    #print(tmp_file)
    cv2.imwrite(tmp_file, output)
    #output = cv2.imread(tmp_file,cv2.IMREAD_COLOR)
    before_time = datetime.datetime.now()
    
    
    # Same as before
    img = full_img
    gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
    gradX = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 1, dy = 0, ksize = -1)
    #gradY = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 0, dy = 1, ksize = -1)
    #gradient = cv2.subtract(gradX, gradY)
    #gradient = cv2.convertScaleAbs(gradient)
    gradient = cv2.convertScaleAbs(gradX)
    
    blurred = cv2.blur(gradient, (9, 9))
    #cv2.imwrite("t1.jpg", blurred)
    closed = cv2.erode(blurred, None, iterations = 4)
    #cv2.imwrite("t2.jpg", closed)
    blurred = cv2.dilate(closed, None, iterations = 4)
    #cv2.imwrite("t3.jpg", blurred)

    (_, cnts, _) = cv2.findContours(blurred.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    img_h,img_w,img_c = img.shape
    min_area,max_area = 0,img_w/2.0*img_h/2
    boxes = []
    #iimg = copy.copy(img)
    for (i, c) in enumerate(cnts):
        (x, y, w, h) = cv2.boundingRect(c)
        if w>0.2*h:
            if w*h>min_area and w*h<max_area: #scale and area filter
                #print(w*h)
                if x>=150 and x <=img_w-150: #position filter
                    if y>=50 and y <=img_h-50:
                        boxes.append((x,y,x+w,y+h))
                        #cv2.rectangle(iimg,(x,y),(x+w,y+h),(0,0,255),2)
    #print(len(boxes))
    boxes = merge_boxes(boxes)
    #print("==============")
    #print(boxes)
    #cv2.imwrite("t4.jpg", iimg)       
    result = 255*np.ones((img_h,img_w,img_c),img.dtype)
    for xmin,ymin,xmax,ymax in boxes:
        if (xmax-xmin)*(ymax-ymin) <= 250:
            continue
        result[ymin:ymax,xmin:xmax,:] = img[ymin:ymax,xmin:xmax,:]
        #cv2.rectangle(result,(xmin,ymin),(xmax,ymax),(0,255,0),2)
    result = parallel_update_img(result, threashold_g-20, threashold_r-20, [255, 255, 255], origin_img_flag=True)

    #cv2.rectangle(result,(xmin,ymin),(xmax,ymax),(0,255,0),2)
    after_time = datetime.datetime.now()
    print("Rest Part Image Spend {}".format((after_time-before_time).seconds))
    
    output_file = "{}/{}_transfer.jpg".format(target_path, os.path.splitext(os.path.basename(source_file))[0])
    cv2.imwrite(output_file,result)
    #break


Processing ZP0699-6cb3417cc48927e7becfb1be0cfa996e.jpg
[104.15263682585316, 244.82142828621625, 139.95946078431373, 203.31357843137255, 168, 84, 57]
9.0 - 12.0
Transfer Image Spend 1
Rest Part Image Spend 2


In [5]:
###########################################3

99.04099158813821 -- 40.334384682957996 -- 11.437388196773679 -- 245.35120188468295


In [12]:
lv_filenames = glob.glob(os.path.join("image/origin-bk", '*.jpg'))
hsv_mean_lst = []
blue_color_lst = []
file_name_lst = []
top_high_lst = []
top_low_lst = []
randmo_lst = []
max_len = 5
for idx, filename in enumerate(lv_filenames):
    full_img = cv2.imread(filename, cv2.IMREAD_COLOR)
    qrcode_file = "{}/{}_qrcode.jpg".format(qrcode_path, os.path.splitext(os.path.basename(filename))[0])
    if not os.path.exists(qrcode_file):
        print("Missing qrcode jpg for image {}".format(source_file))
        continue
    qrcode_img = cv2.imread(qrcode_file,cv2.IMREAD_COLOR)
    file_name_lst.append(os.path.basename(filename))    
    blue_color = get_blue_color_center_by_kmeans(qrcode_img, 2)
    blue_color_lst.append(blue_color)
    full_mean, full_v_mean = get_hsv_stat(full_img)
    qr_mean, qr_v_mean = get_hsv_stat(qrcode_img)
    hsv_mean_lst.append([full_mean, full_v_mean, qr_mean, qr_v_mean])
    
    fill_flag_high = False
    if len(top_high_lst) == 0:
        top_high_lst.append([idx, np.mean([full_mean, full_v_mean, qr_mean, qr_v_mean])])
    else:
        
        for tmp_idx, values in enumerate(top_high_lst):
            if np.mean([full_mean, full_v_mean, qr_mean, qr_v_mean])> values[1]:
                top_high_lst.insert(tmp_idx, [idx, np.mean([full_mean, full_v_mean, qr_mean, qr_v_mean])])
                fill_flag_high = True
                break
        if not fill_flag_high:
            top_high_lst.append([idx, np.mean([full_mean, full_v_mean, qr_mean, qr_v_mean])])
        if len(top_high_lst) > max_len:
            del top_high_lst[-1]
    
    fill_flag_low = False
    if len(top_low_lst) == 0:
        top_low_lst.append([idx, np.mean([full_mean, full_v_mean, qr_mean, qr_v_mean])])
    else:  
        
        for tmp_idx, values in enumerate(top_low_lst):
            if np.mean([full_mean, full_v_mean, qr_mean, qr_v_mean]) < values[1]:
                top_low_lst.insert(tmp_idx, [idx, np.mean([full_mean, full_v_mean, qr_mean, qr_v_mean])])
                fill_flag_low = True
                break
        if not fill_flag_low:
            top_low_lst.append([idx, np.mean([full_mean, full_v_mean, qr_mean, qr_v_mean])])
        if len(top_low_lst) > max_len:
            del top_low_lst[-1]

    if len(randmo_lst) < max_len and (not fill_flag_high) and len(top_high_lst) == max_len and (not fill_flag_low) and len(top_low_lst) == max_len:
        randmo_lst.append([idx, np.mean([full_mean, full_v_mean, qr_mean, qr_v_mean])])
print(top_high_lst[:3])
        
with open("record.txt", "w") as f:
    for idx, filename in enumerate(file_name_lst):
        content = "{}: {} {}\n".format(os.path.basename(filename), hsv_mean_lst[idx], blue_color_lst[idx])
        f.write(content)
with open("top_high_record.txt", "w") as f:
    for values in top_high_lst:
        content = "{}: {} {} {}\n".format(os.path.basename(file_name_lst[values[0]]), hsv_mean_lst[values[0]], blue_color_lst[values[0]], values[1])
        f.write(content)
with open("top_low_record.txt", "w") as f:
    for values in top_low_lst:
        content = "{}: {} {} {}\n".format(os.path.basename(file_name_lst[values[0]]), hsv_mean_lst[values[0]], blue_color_lst[values[0]], values[1])
        f.write(content)
with open("random_record.txt", "w") as f:
    for values in randmo_lst:
        content = "{}: {} {} {}\n".format(os.path.basename(file_name_lst[values[0]]), hsv_mean_lst[values[0]], blue_color_lst[values[0]], values[1])
        f.write(content)


[[166, 184.83650402029775], [41, 184.7918762239171], [94, 182.97666992789735]]


In [19]:
model_g, model_r = init_predict_mode()
feature = [111.70328807964648, 239.25448690305063, 142.53413333333333, 179.79308571428572, 142,  36,  29]
threshold_g = round(model_g.predict([feature])[0])
threshold_r = round(model_r.predict([feature])[0])
print("{} {}".format(threshold_g, threshold_r))

19.0 37.0


In [13]:
boxA = [[10, 10, 50, 50], [12, 13, 20, 25], [100, 140, 220, 400], [52, 12, 100, 30]]
boxB = merge_boxes(boxA)
print(boxB)


[[10, 10, 100, 50], array([100, 140, 220, 400])]
